#  BI-LSTM
- BI-LSTM은 정보를 역방향으로 전달하는 히든 레이어를 추가하여 이러한 정보를 보다 유연하게 처리

In [1]:
import os
import time
import numpy as np
from tqdm import tqdm
from string import punctuation  # 구두점 제거에 이용
from collections import Counter # 데이터의 개수를 셀 때 유용한 클래스
import matplotlib.pyplot as plt

import torch
# 신경망 생성 및 학습에 도움을 주는 모듈/클래스
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset # 데이터셋을 불러온 뒤, 순회할 수 있는 객체
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # GPU 가능시 device='cuda'
torch.manual_seed(123) # random seed를 고정함

In [2]:
device

device(type='cuda')

## 텍스트 데이터셋 로딩과 전처리

In [3]:
import random
from torchtext import (data, datasets) # 텍스트 분류 분석을 위한 데이터셋

In [4]:
# 필드를 통해 앞으로 어떤 전처리를 할 것인지를 정의합니다
TEXT_FIELD = data.Field(tokenize = data.get_tokenizer("basic_english"), include_lengths = True)
LABEL_FIELD = data.LabelField(dtype = torch.float)

# 리뷰텍스트와 감성레이블 두개의 필드로 나눈다
train_dataset, test_dataset = datasets.IMDB.splits(TEXT_FIELD, LABEL_FIELD) 
train_dataset, valid_dataset = train_dataset.split(random_state = random.seed(123))

 - torchtext.data.Field와 torchtext.data.LabelField의 build_vocab 메서드를 사용해 <br>각각 영화 리뷰 텍스트 데이터셋(IMDb)과 감성 레이블에 대한 사전을 구성한다.

In [5]:
MAX_VOCABULARY_SIZE = 25000

# build_vocab 메서드로 단어사전을 구성
TEXT_FIELD.build_vocab(train_dataset, max_size = MAX_VOCABULARY_SIZE)  
LABEL_FIELD.build_vocab(train_dataset)

**Bucketing은 주어진 문장의 길이에 따라 데이터를 그룹화하여 padding을 적용하는 기법이다**

- 길이가 천차만별인 데이터들을 하나의 batch내에 넣는다면 가장 큰 데이터의 길이만큼  <br>padding이 되어야하므로 쓸데없이 0으로 차있게 돼 학습에 시간이 오래걸린다. <br><br>

- 하지만 Bucketing은 길이가 비슷한 데이터들끼리 하나의 batch로 만들어 padding을 최소화시킨다. <br><br>

- 이 기법은 모델의 학습 시간을 단축하기 위해 고안되었다.

**Bucketiterator**
- Pytorch의 dataloader와 비슷한 역할을 한다. <br><br>
- 하지만 dataloader 와 다르게 비슷한 길이의 문장들끼리 batch를 만들기 때문에<br> padding의 개수를 최소화할 수 있다.

In [6]:
B_SIZE = 64  # batch size

train_data_iterator, valid_data_iterator, test_data_iterator = \
data.BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset), 
    batch_size = B_SIZE,
    sort_within_batch = True,
    device = device)

**PackedSequence**
- NLP에서 매 배치마다 고정된 문장의 길이로 만들어주기 위해서 <pad.> 토큰을 넣어야하는데, <br>
문장의 길이별로 정렬해주지 않고 수행을 하면 <pad.> 토큰까지 연산을 하게 된다. <br>
따라서 이를 계산하지않고 효율적으로 진행하기 위해 병렬처리를 하려고한다. <br>
이를 통해 sequence를 마치 상삼각행렬을 좌우반전 시킨형태로 정렬시킨다.

In [7]:
# GPU 환경을 사용하는 경우 pack_padded_sequence 메서드가 작동하려면 다음 함수를 사용 
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
from torch.nn.utils.rnn import pack_padded_sequence, PackedSequence

def cuda_pack_padded_sequence(input, lengths, batch_first=False, enforce_sorted=True):
    lengths = torch.as_tensor(lengths, dtype=torch.int64)
    lengths = lengths.cpu()
    if enforce_sorted:
        sorted_indices = None
    else:
        lengths, sorted_indices = torch.sort(lengths, descending=True)
        sorted_indices = sorted_indices.to(input.device)
        batch_dim = 0 if batch_first else 1
        input = input.index_select(batch_dim, sorted_indices)

    data, batch_sizes = \
    torch._C._VariableFunctions._pack_padded_sequence(input, lengths, batch_first)
    return PackedSequence(data, batch_sizes, sorted_indices)

## LSTM 모델 인스턴스화 및 훈련

**nn.Embedding**
- *num_embeddings* : 임베딩을 할 단어들의 개수. 다시 말해 **단어 집합의 크기**입니다.<br><br>
- *embedding_dim* : **임베딩 할 벡터의 차원**입니다. 사용자가 정해주는 하이퍼파라미터입니다.<br><br>
- *padding_idx* : 선택적으로 사용하는 인자입니다. **패딩을 위한 토큰의 인덱스**를 알려줍니다.

**nn.LSTM**
- *input_size* -- **The number of expected features in the input x**<br><br>

- *hidden_size* -- **The number of features in the hidden state h**<br><br>

- *num_layers* -- **Number of recurrent layers**. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM, <br>with the second LSTM taking in outputs of the first LSTM and computing the final results.<br> Default: 1<br><br>

- *bias* -- **If False**, then the layer **does not use bias weights b_ih and b_hh**.<br> Default: True<br><br>

- *batch_first* -- **If True**, then the **input and output tensors are provided as (batch, seq, feature)** instead of (seq, batch, feature). <br>Note that this does not apply to hidden or cell states. See the Inputs/Outputs sections below for details. Default: False<br><br>

- *dropout* -- **If non-zero**, **introduces a Dropout layer on the outputs of each LSTM layer except the last layer**, with dropout probability <br>equal to dropout.<br> Default: 0<br><br>

- *bidirectional* -- **If True**, **becomes a bidirectional LSTM**.<br> Default: False<br><br>

- *proj_size* -- **If > 0**, will use LSTM with **projections of corresponding size**.<br> Default: 0<br><br><br>
from pytorch document

In [8]:
class LSTM(nn.Module):
    def __init__(self, vocabulary_size, embedding_dimension, hidden_dimension, output_dimension, dropout, pad_index):
        super().__init__()
        # 임베딩 테이블을 생성합니다, embedding_dimension만큼 축소
        self.embedding_layer = nn.Embedding(vocabulary_size, embedding_dimension, padding_idx = pad_index)
        self.lstm_layer = nn.LSTM(embedding_dimension, 
                                  hidden_dimension, 
                                  num_layers=1,  # 재귀 층의 개수, stacked lstm의 경우에는 2개 이상
                                  bidirectional=True,  # numdirection=2
                                  dropout=dropout)
        self.fc_layer = nn.Linear(hidden_dimension * 2, output_dimension)
        self.dropout_layer = nn.Dropout(dropout)
        
    def forward(self, sequence, sequence_lengths=None):
        if sequence_lengths is None:
            sequence_lengths = torch.LongTensor([len(sequence)]) # 64비트의 부호 있는 정수는 torch.LongTensor를 사용합니다
        
        # sequence := (sequence_length, batch_size)
        embedded_output = self.dropout_layer(self.embedding_layer(sequence))
        
        
        # embedded_output := (sequence_length, batch_size, embedding_dimension)
        # GPU 환경 사용시 위에서 정의한 함수를 사용
        if torch.cuda.is_available():  
            # PackedSequence object를 얻는다
            packed_embedded_output = cuda_pack_padded_sequence(embedded_output, sequence_lengths) 
        else:
            # PackedSequence object를 얻는다
            packed_embedded_output = nn.utils.rnn.pack_padded_sequence(embedded_output, sequence_lengths)
        
        packed_output, (hidden_state, cell_state) = self.lstm_layer(packed_embedded_output)
        # hidden_state := (num_layers * num_directions, batch_size, hidden_dimension)
        # cell_state := (num_layers * num_directions, batch_size, hidden_dimension)
        
        op, op_lengths = nn.utils.rnn.pad_packed_sequence(packed_output) # 패킹된 문장을 다시 unpack
        # op := (sequence_length, batch_size, hidden_dimension * num_directions)
        
        hidden_output = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1) 
        # bidirectional=True이므로 이전 hidden state 2개를 호출하여 연결 후 사용한다
        # hidden_output := (batch_size, hidden_dimension * num_directions)
        
        return self.fc_layer(hidden_output)

    


In [9]:
INPUT_DIMENSION = len(TEXT_FIELD.vocab)
EMBEDDING_DIMENSION = 100
HIDDEN_DIMENSION = 32
OUTPUT_DIMENSION = 1
DROPOUT = 0.5
PAD_INDEX = TEXT_FIELD.vocab.stoi[TEXT_FIELD.pad_token]

lstm_model = LSTM(INPUT_DIMENSION, 
            EMBEDDING_DIMENSION, 
            HIDDEN_DIMENSION, 
            OUTPUT_DIMENSION, 
            DROPOUT, 
            PAD_INDEX)

C:\Users\ebdl\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


- 사전에 두개의 특수 토큰 추가<br>
- 하나는 사전에 없는 단어를 위한 unknown토큰,다른 하나는 시퀀스 패딩을 위해 추가디는 padding 토큰이다

In [10]:
UNK_INDEX = TEXT_FIELD.vocab.stoi[TEXT_FIELD.unk_token] # 현재 단어 집합의 단어와 맵핑된 고유한 정수를 출력할 수 있다

lstm_model.embedding_layer.weight.data[UNK_INDEX] = torch.zeros(EMBEDDING_DIMENSION) # 가중치 초기화
lstm_model.embedding_layer.weight.data[PAD_INDEX] = torch.zeros(EMBEDDING_DIMENSION) # 가중치 초기화

In [11]:
optim = torch.optim.Adam(lstm_model.parameters()) # optimizer = 'Adam'
# BCELoss에서는 CrossEntropyLoss와 같이 softmax를 포함한 것이 아닌, Cross Entropy만 구합니다
# (Sigmoid + BCELoss) 따라서 따로 sigmoid 나 softmax를 해줄 필요가 없습니다
loss_func = nn.BCEWithLogitsLoss()

lstm_model = lstm_model.to(device)
loss_func = loss_func.to(device)

In [12]:
def accuracy_metric(predictions, ground_truth):
    """
    Returns 0-1 accuracy for the given set of predictions and ground truth
    """
    # 예측을 0 또는 1로 반올림
    rounded_predictions = torch.round(torch.sigmoid(predictions))
    success = (rounded_predictions == ground_truth).float() # 나눗셈을 위해 float로 변환
    accuracy = success.sum() / len(success)
    return accuracy

In [13]:
def train(model, data_iterator, optim, loss_func):
    loss = 0
    accuracy = 0
    model.train()
    
    for curr_batch in data_iterator:
        optim.zero_grad()
        sequence, sequence_lengths = curr_batch.text
        preds = lstm_model(sequence, sequence_lengths).squeeze(1)
        
        loss_curr = loss_func(preds, curr_batch.label)
        accuracy_curr = accuracy_metric(preds, curr_batch.label)
        
        loss_curr.backward()
        optim.step()
        
        loss += loss_curr.item()
        accuracy += accuracy_curr.item()
        
    return loss/len(data_iterator), accuracy/len(data_iterator)

'''
model=lstm
data_iterator=train_data_iterator
optim=Adam
loss_func=BCEWithLogitsLoss
'''

'\nmodel=lstm\ndata_iterator=train_data_iterator\noptim=Adam\nloss_func=BCEWithLogitsLoss\n'

In [14]:
def validate(model, data_iterator, loss_func):
    loss = 0
    accuracy = 0
    model.eval()
    
    with torch.no_grad():
        for curr_batch in data_iterator:
            sequence, sequence_lengths = curr_batch.text
            preds = model(sequence, sequence_lengths).squeeze(1)
            
            loss_curr = loss_func(preds, curr_batch.label)
            accuracy_curr = accuracy_metric(preds, curr_batch.label)

            loss += loss_curr.item()
            accuracy += accuracy_curr.item()
        
    return loss/len(data_iterator), accuracy/len(data_iterator)

In [15]:
num_epochs = 10
best_validation_loss = float('inf')

for ep in range(num_epochs):

    time_start = time.time()
    
    training_loss, train_accuracy = train(lstm_model, train_data_iterator, optim, loss_func)
    validation_loss, validation_accuracy = validate(lstm_model, valid_data_iterator, loss_func)
    
    time_end = time.time()
    time_delta = time_end - time_start 
    
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        torch.save(lstm_model.state_dict(), 'lstm_model.pt')
    
    print(f'epoch number: {ep+1} | time elapsed: {time_delta}s')
    print(f'training loss: {training_loss:.3f} | training accuracy: {train_accuracy*100:.2f}%')
    print(f'validation loss: {validation_loss:.3f} |  validation accuracy: {validation_accuracy*100:.2f}%')
    print()

epoch number: 1 | time elapsed: 5.629080057144165s
training loss: 0.687 | training accuracy: 55.23%
validation loss: 0.669 |  validation accuracy: 58.54%

epoch number: 2 | time elapsed: 5.211093425750732s
training loss: 0.657 | training accuracy: 61.17%
validation loss: 0.868 |  validation accuracy: 58.99%

epoch number: 3 | time elapsed: 5.304181814193726s
training loss: 0.580 | training accuracy: 69.28%
validation loss: 0.751 |  validation accuracy: 63.22%

epoch number: 4 | time elapsed: 5.317663669586182s
training loss: 0.516 | training accuracy: 75.04%
validation loss: 0.764 |  validation accuracy: 69.41%

epoch number: 5 | time elapsed: 5.233748912811279s
training loss: 0.466 | training accuracy: 78.37%
validation loss: 0.650 |  validation accuracy: 71.18%

epoch number: 6 | time elapsed: 5.318321228027344s
training loss: 0.433 | training accuracy: 80.42%
validation loss: 0.622 |  validation accuracy: 75.88%

epoch number: 7 | time elapsed: 5.384981155395508s
training loss: 0.40

훈련과 검증셋의 정확도가 비슷한 속도로 증가하는 것으로 보아 드롭아웃이 과적합을 제어하는 것으로 보임

In [16]:
# 이전 단계에서 가장 성능이 좋은 모델을 저장, 이 모델을 로딩해서 테스트셋에서 검증
lstm_model.load_state_dict(torch.load('lstm_model.pt'))

test_loss, test_accuracy = validate(lstm_model, test_data_iterator, loss_func)

print(f'test loss: {test_loss:.3f} | test accuracy: {test_accuracy*100:.2f}%')

test loss: 0.499 | test accuracy: 80.45%


In [17]:
# 감성 추론 함수를 정의
def sentiment_inference(model, sentence):
    model.eval()
    
    # text transformations
    tokenized = data.get_tokenizer("basic_english")(sentence)
    tokenized = [TEXT_FIELD.vocab.stoi[t] for t in tokenized]
    
    # model inference
    model_input = torch.LongTensor(tokenized).to(device)
    model_input = model_input.unsqueeze(1)
    
    pred = torch.sigmoid(model(model_input))
    
    return pred.item()

긍정은 1에 가깝게 부정은 0에 가깝게

In [18]:
print(sentiment_inference(lstm_model, "This film is horrible"))
print(sentiment_inference(lstm_model, "Director tried too hard but this film is bad"))
print(sentiment_inference(lstm_model, "Decent movie, although could be shorter"))
print(sentiment_inference(lstm_model, "This film will be houseful for weeks"))
print(sentiment_inference(lstm_model, "I loved the movie, every part of it"))

0.0925179272890091
0.00595007324591279
0.07338246703147888
0.5898579955101013
0.9808263778686523
